In [5]:

!pip install openeo shapely rasterio opencv-python-headless kaggle --quiet

import os
import cv2
import csv
import json
import rasterio
import numpy as np
from shapely.geometry import box
import openeo




from google.colab import files
files.upload()  # Upload kaggle.json
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d shambac/tum-sentinel-1-2 --unzip -p sentinel_data

# CONNECT TO COPERNICUS OPENEO
connection = openeo.connect("openeo.dataspace.copernicus.eu")
connection.authenticate_oidc()

# MATCH METHOD
def match_images(img1_path, img2_path):
    img1 = cv2.imread(img1_path, cv2.IMREAD_GRAYSCALE)
    img2 = cv2.imread(img2_path, cv2.IMREAD_GRAYSCALE)
    orb = cv2.ORB_create()
    kp1, des1 = orb.detectAndCompute(img1, None)
    kp2, des2 = orb.detectAndCompute(img2, None)
    if des1 is None or des2 is None:
        return 0
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(des1, des2)
    return len(matches)

#EXRACTING COORDINATES AND CALCULATING REGION
def extract_coordinates(geotiff_path):
    with rasterio.open(geotiff_path) as dataset:
        bounds = dataset.bounds
        return bounds.top, bounds.left, bounds.bottom, bounds.right

def get_hemisphere(lat):
    return "Northern Hemisphere" if lat >= 0 else "Southern Hemisphere"

def get_temperature_zone(lat):
    if abs(lat) < 23.5:
        return "Tropical Zone"
    elif abs(lat) < 66.5:
        return "Temperate Zone"
    else:
        return "Polar Zone"

def download_tiff_from_bbox(bbox, date_range, filename):
    cube = connection.load_collection(
        "SENTINEL2_L2A",
        spatial_extent={
            "west": bbox.bounds[0],
            "south": bbox.bounds[1],
            "east": bbox.bounds[2],
            "north": bbox.bounds[3],
            "crs": "EPSG:4326"
        },
        temporal_extent=date_range,
        bands=["B04", "B03", "B02"]
    ).reduce_dimension(dimension="t", reducer="median")
    cube.download(filename, format="GTiff")

def download_png_from_bbox(bbox, date_range, filename):
    cube = connection.load_collection(
        "SENTINEL2_L2A",
        spatial_extent={
            "west": bbox.bounds[0],
            "south": bbox.bounds[1],
            "east": bbox.bounds[2],
            "north": bbox.bounds[3],
            "crs": "EPSG:4326"
        },
        temporal_extent=date_range,
        bands=["B04", "B03", "B02"]
    ).reduce_dimension(dimension="t", reducer="median")
    cube.download(filename, format="PNG")

# DEFINE SEASON DATE RANGES
seasons = {
    "Sentinel_fall":   ["2017-09-01", "2017-11-30"],
    "Sentinel_spring": ["2017-03-01", "2017-05-30"],
    "Sentinel_summer": ["2017-06-01", "2017-08-31"],
    "Sentinel_winter": ["2016-12-01", "2017-02-28"]
}

# MAIN PROCESSING LOOP
data_root = "sentinel_data"
output_root = "output_csvs"
os.makedirs(output_root, exist_ok=True)

for season_folder in sorted(seasons.keys()):
    full_season_path = os.path.join(data_root, season_folder)
    if not os.path.isdir(full_season_path):
        continue

    print(f" Processing season: {season_folder}")

    for region_folder in os.listdir(full_season_path):
        region_path = os.path.join(full_season_path, region_folder)
        if not os.path.isdir(region_path):
            continue

        for target_img_name in os.listdir(region_path):
            if not target_img_name.endswith(".png"):
                continue

            target_path = os.path.join(region_path, target_img_name)

            best_score = 0
            best_bbox = None

            # Dummy bbox from ROI latitude
            try:
                lat = float(region_folder.replace("ROI", ""))
            except ValueError:
                print(f"Skipping invalid region name: {region_folder}")
                continue
            lon = 77.6
            bbox = box(lon - 0.05, lat - 0.05, lon + 0.05, lat + 0.05)

            # Download PNG for matching
            try:
                temp_png = f"temp_{season_folder}_{region_folder}.png"
                download_png_from_bbox(bbox, seasons[season_folder], temp_png)
                score = match_images(target_path, temp_png)

                if score > best_score:
                    best_score = score
                    best_bbox = bbox

                os.remove(temp_png)

            except Exception as e:
                print(f"Error while matching {target_img_name}: {e}")
                continue

            # Download TIFF & log results
            if best_bbox:
                tiff_file = f"{season_folder}_{region_folder}.tif"
                download_tiff_from_bbox(best_bbox, seasons[season_folder], tiff_file)

                top, left, bottom, right = extract_coordinates(tiff_file)
                hemisphere = get_hemisphere(top)
                temp_zone = get_temperature_zone(top)

                season_csv_dir = os.path.join(output_root, season_folder)
                os.makedirs(season_csv_dir, exist_ok=True)
                csv_path = os.path.join(season_csv_dir, "match_output.csv")

                with open(csv_path, mode="a", newline="") as f:
                    writer = csv.writer(f)
                    if os.stat(csv_path).st_size == 0:
                        writer.writerow(["s1_filename", "temperature_zone", "hemisphere", "latitude"])
                    writer.writerow([target_img_name, temp_zone, hemisphere, round(top, 3)])

                os.remove(tiff_file)
                print(f"Processed {target_img_name} in {season_folder}/{region_folder}")

print("All seasons processed!")


Saving kaggle.json to kaggle (4).json
Dataset URL: https://www.kaggle.com/datasets/shambac/tum-sentinel-1-2
License(s): CC-BY-SA-4.0
User cancelled operation
Authenticated using refresh token.
All seasons processed!
